In [2]:
# @title Setup for Local Environment

# @title libraries
# Standard library
from pathlib import Path
import os
import re
import json
import datetime
import logging
import warnings

# Data handling
import pandas as pd
import numpy as np

# Excel & HTML parsing
from openpyxl import load_workbook, Workbook
import xlrd       # Older XLS engine
import lxml       # HTML/XML parser
import html5lib   # HTML parser alternative
from bs4 import BeautifulSoup

# Config files
import yaml       # For reading config.yaml

# Image processing
from PIL import Image

# Template rendering for LaTeX
from jinja2 import Environment, FileSystemLoader

# Display
from IPython.display import display

# @title define directories

# Set up local paths
PROJECT_ROOT = Path("/home/ben/Desktop/Projects/media_guide")
DATA_DIR = PROJECT_ROOT / "latex" / "code" / "data"

# Create output directory for processed data
OUT_FOLDER = PROJECT_ROOT / "processed_data"
OUT_FOLDER.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUT_FOLDER}")
print(f"Data files found: {list(DATA_DIR.glob('*.xlsx'))}")

Project root: /home/ben/Desktop/Projects/media_guide
Data directory: /home/ben/Desktop/Projects/media_guide/latex/code/data
Output directory: /home/ben/Desktop/Projects/media_guide/processed_data
Data files found: []


In [3]:
# @title Check Available Data Files

# List all Excel files in the data directory
excel_files = list(DATA_DIR.glob("*.xlsx"))
print(f"Found {len(excel_files)} Excel files:")
for i, file in enumerate(excel_files, 1):
    print(f"{i}. {file.name}")

print(f"\nData directory: {DATA_DIR}")
print(f"Output directory: {OUT_FOLDER}")

# Check if output directory exists and is writable
if OUT_FOLDER.exists():
    print(f"✅ Output directory exists: {OUT_FOLDER}")
else:
    print(f"❌ Output directory does not exist: {OUT_FOLDER}")
    OUT_FOLDER.mkdir(parents=True, exist_ok=True)
    print(f"✅ Created output directory: {OUT_FOLDER}")


Found 0 Excel files:

Data directory: /home/ben/Desktop/Projects/media_guide/latex/code/data
Output directory: /home/ben/Desktop/Projects/media_guide/processed_data
✅ Output directory exists: /home/ben/Desktop/Projects/media_guide/processed_data


In [ ]:
# @title Process Excel Files to Master CSV

def process_excel_to_master_csv():
    """Process all Excel files and create a master CSV with standardized format"""
    
    all_data = []
    
    for excel_file in excel_files:
        print(f"Processing: {excel_file.name}")
        
        try:
            # Load the Excel file
            xls = pd.ExcelFile(excel_file)
            
            for sheet_name in xls.sheet_names:
                print(f"  Sheet: {sheet_name}")
                
                # Read the sheet
                df = pd.read_excel(excel_file, sheet_name=sheet_name, header=None)
                
                # Add metadata columns
                df['SOURCE_FILE'] = excel_file.name
                df['SHEET_NAME'] = sheet_name
                
                # Add to all_data
                all_data.append(df)
                
        except Exception as e:
            print(f"  Error processing {excel_file.name}: {e}")
            continue
    
    if all_data:
        # Combine all dataframes
        master_df = pd.concat(all_data, ignore_index=True)
        
        # Save to CSV
        master_csv_path = OUT_FOLDER / "master_data.csv"
        master_df.to_csv(master_csv_path, index=False)
        
        print(f"\n✅ Created master CSV: {master_csv_path}")
        print(f"Total rows: {len(master_df)}")
        print(f"Columns: {list(master_df.columns)}")
        
        return master_df
    else:
        print("❌ No data processed")
        return pd.DataFrame()

# Uncomment the line below to process your Excel files
# master_df = process_excel_to_master_csv()


In [ ]:
# @title functions

def save_clean_df(df_clean, file_path, sheet_name, out_folder):
    """Save cleaned DataFrame with safe filename."""
    safe_sheet = sheet_name.replace(" ", "_").replace("/", "_")
    safe_file = file_path.stem.replace(" ", "_").replace("/", "_")
    out_path = out_folder / f"{safe_file}_{safe_sheet}_clean.csv"
    df_clean.to_csv(out_path, index=False)
    print(f"Saved cleaned data to {out_path}")

In [ ]:
# @title test file

# File path
file_path = DATA_DIR / "SCIAC Top 10 4.1.25.xlsx"

# Inspect sheet names
xls = pd.ExcelFile(file_path)
print("Sheet names:", xls.sheet_names)

# Load the first sheet (index 0)
sheet_name = xls.sheet_names[0]
df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Show first 20 rows so we can decide cleaning steps
df.head(5)

#save_clean_df(df_clean, file_path, sheet_name, OUT_FOLDER)

https://cmsathletics.org/services/schedule_txt.ashx?schedule=365
https://cmsathletics.org/services/schedule_txt.ashx?schedule=382
https://cmsathletics.org/services/schedule_txt.ashx?schedule=381
https://cmsathletics.org/services/schedule_txt.ashx?schedule=364

In [ ]:
# @title Merge

SRC_DIR = DATA_DIR  # Use the DATA_DIR we defined earlier
OUT_PATH = OUT_FOLDER / "merged.xlsx"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

wb_out = Workbook()
wb_out.remove(wb_out.active)  # drop default empty sheet

for f in SRC_DIR.glob("*.xlsx"):
    try:
        wb_in = load_workbook(f, data_only=True)
    except Exception as e:
        print(f"Skipping {f.name}: {e}")
        continue

    for ws in wb_in.worksheets:
        # make a unique name (Excel caps at 31 chars)
        out_name = (f"{f.stem}_{ws.title}")[:31]
        out_ws = wb_out.create_sheet(out_name)
        for row in ws.iter_rows(values_only=True):
            out_ws.append(row)

    wb_in.close()

wb_out.save(OUT_PATH)
print(f"✅ Saved merged workbook to {OUT_PATH}")

/usr/local/lib/python3.12/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


✅ Saved merged workbook to /content/drive/My Drive/cms_media_guide/processed_data/merged.xlsx


In [ ]:
# Check if the CSV file exists, if not, we'll need to create it from the Excel files
csv_file = OUT_FOLDER / "master_CMS - times-2.csv"
if csv_file.exists():
    times_df = pd.read_csv(csv_file)
    print(f"Loaded existing CSV file: {csv_file}")
else:
    print(f"CSV file not found: {csv_file}")
    print("You may need to create this file from your Excel data first.")
    # Create an empty dataframe for now
    times_df = pd.DataFrame()

times_df.head()

,SHEET,SEX,EVENT,TIME,NAME,YEAR,TEAM,RANK,SITE,MEET,CONTEXT
0,CMS at PP,Athena,50 FREE,23.29,Jocelyn Crawford,2018,NaN,NaN,NaN,NaN,NaN
1,CMS at PP,Athena,50 FREE,23.64,Jocelyn Crawford,2017,NaN,NaN,NaN,NaN,NaN
2,CMS at PP,Athena,50 FREE,23.66,Madeleine Kan,2024,NaN,NaN,NaN,NaN,NaN
3,CMS at PP,Athena,50 FREE,23.89,Ava Sealander,2021,NaN,NaN,NaN,NaN,NaN
4,CMS at PP,Athena,50 FREE,24.04,Ava Sealander,2019,NaN,NaN,NaN,NaN,NaN


In [7]:
# Strip whitespace from column names just in case
times_df.columns = times_df.columns.str.strip()

# Columns to ignore
ignore_cols = {"SHEET", "SEX", "EVENT"}

# Create a mask of non-null columns (excluding ignored ones)
patterns = times_df.drop(columns=ignore_cols, errors="ignore").notna()

# Turn each row’s pattern into a tuple of filled-in column names
filled_sets = patterns.apply(
    lambda row: tuple(col for col, filled in row.items() if filled),
    axis=1
)

# Get unique combinations
unique_combinations = filled_sets.unique()

print("Unique combinations of filled columns (ignoring SHEET and SEX):")
for combo in unique_combinations:
    print(combo)


Unique combinations of filled columns (ignoring SHEET and SEX):
('TIME', 'NAME', 'YEAR')
('TIME', 'NAME', 'YEAR', 'TEAM')
('TIME', 'NAME', 'YEAR', 'TEAM', 'RANK')
('TIME', 'YEAR', 'TEAM', 'RANK')
('NAME', 'YEAR')
('TIME', 'NAME', 'YEAR', 'SITE')
('TIME', 'NAME', 'YEAR', 'SITE', 'MEET', 'CONTEXT')
('TIME', 'NAME', 'YEAR', 'TEAM', 'MEET')


In [11]:
latex_labels = {
    "TIME": "Time",
    "NAME": "Name",
    "YEAR": "Year",
    "TEAM": "Team",
    "RANK": "Rank",
    "SITE": "Site",
    "MEET": "Meet",
    "CONTEXT": "Context"
}


# Assuming `unique_combinations` already exists from previous step

macros = []
for combo in unique_combinations:
    # Macro name based on joined column names
    macro_name = "eventtable" + "".join(col.capitalize() for col in combo)

    # Column format string: l for text, r for numbers (here: YEAR and RANK)
    col_format = " ".join("r" if col in {"YEAR", "RANK"} else "l" for col in combo)

    # Table headers
    headers = " & ".join(f"\\textbf{{{latex_labels[col]}}}" for col in combo)

    macro = f"""
% Auto-generated macro
\\newcommand{{\\{macro_name}}}[2]{{
\\begin{{table}}[h!]
\\centering
\\caption*{{#1}}
\\vspace{{-0.25cm}}
\\begin{{tabular}}{{{col_format}}}
\\hline
{headers} \\\\
\\hline
#2
\\hline
\\end{{tabular}}
\\end{{table}}
}}
"""
    macros.append(macro)

for macro in macros:
    print(macro)



% Auto-generated macro
\newcommand{\eventtableTimeNameYear}[2]{
\begin{table}[h!]
\centering
\caption*{#1}
\vspace{-0.25cm}
\begin{tabular}{l l r}
\hline
\textbf{Time} & \textbf{Name} & \textbf{Year} \\
\hline
#2
\hline
\end{tabular}
\end{table}
}


% Auto-generated macro
\newcommand{\eventtableTimeNameYearTeam}[2]{
\begin{table}[h!]
\centering
\caption*{#1}
\vspace{-0.25cm}
\begin{tabular}{l l r l}
\hline
\textbf{Time} & \textbf{Name} & \textbf{Year} & \textbf{Team} \\
\hline
#2
\hline
\end{tabular}
\end{table}
}


% Auto-generated macro
\newcommand{\eventtableTimeNameYearTeamRank}[2]{
\begin{table}[h!]
\centering
\caption*{#1}
\vspace{-0.25cm}
\begin{tabular}{l l r l r}
\hline
\textbf{Time} & \textbf{Name} & \textbf{Year} & \textbf{Team} & \textbf{Rank} \\
\hline
#2
\hline
\end{tabular}
\end{table}
}


% Auto-generated macro
\newcommand{\eventtableTimeYearTeamRank}[2]{
\begin{table}[h!]
\centering
\caption*{#1}
\vspace{-0.25cm}
\begin{tabular}{l r l r}
\hline
\textbf{Time} & \textbf

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Strip whitespace from column names just in case
times_df.columns = times_df.columns.str.strip()

# Prepare unique values
sheets = sorted(times_df["SHEET"].dropna().unique())
sexes = sorted(times_df["SEX"].dropna().unique())

# Dropdowns
sheet_dd = widgets.Dropdown(options=sheets, description="Sheet:")
sex_dd = widgets.Dropdown(options=sexes, description="Sex:")

# Grid controls
rows_dd = widgets.IntSlider(value=3, min=1, max=5, description="Rows:")
cols_dd = widgets.IntSlider(value=3, min=1, max=5, description="Cols:")

# Button
print_btn = widgets.Button(description="Print All Events")

# Sorting key function for EVENT_CLEAN
stroke_order = ["FREE", "BACK", "BREAST", "FLY", "IM"]

def event_sort_key(event):
    event_upper = event.upper()

    # Diving events
    if "METER" in event_upper:
        stroke_index = 999
        match = re.search(r'\d+', event_upper)
        distance = int(match.group()) if match else 0
        dives_match = re.search(r'\((\d+)\s*DIVES\)', event_upper)
        dives = int(dives_match.group(1)) if dives_match else 0
        return (stroke_index, distance, dives, event_upper)

    # Relay/Splits
    relay_flag = 1 if "RELAY" in event_upper or "SPL" in event_upper else 0

    # Individual strokes
    stroke_index = 0
    for i, stroke in enumerate(stroke_order):
        if stroke in event_upper:
            stroke_index = i
            break

    # Numeric distance
    match = re.search(r'\d+', event_upper)
    distance = int(match.group()) if match else 0

    return (relay_flag, stroke_index, distance, event_upper)

# Function to print tables
def print_all_events(_):
    clear_output(wait=True)
    display(sheet_dd, sex_dd, rows_dd, cols_dd, print_btn)

    sheet = sheet_dd.value
    sex = sex_dd.value
    nrows, ncols = rows_dd.value, cols_dd.value

    # Filter dataframe
    df_filtered = times_df[(times_df["SHEET"] == sheet) & (times_df["SEX"] == sex)]
    events = sorted(df_filtered["EVENT_CLEAN"].dropna().unique(), key=event_sort_key)

    if not events:
        print("⚠️ No events found for this sheet and sex.")
        return

    out = ["\\begin{center}"]

    for i, event in enumerate(events):
        df_event = df_filtered[df_filtered["EVENT_CLEAN"] == event]
        cols_used = [col for col in df_event.columns if col not in {"SHEET","SEX","EVENT","EVENT_CLEAN"} and df_event[col].notna().any()]
        macro_name = "eventtable" + "".join(col.capitalize() for col in cols_used)

        # Build table rows
        rows = []
        for _, row in df_event.iterrows():
            vals = [str(row[c]) for c in cols_used]
            rows.append(" & ".join(vals) + " \\\\")
        rows_block = "\n".join(rows)
        latex_cmd = f"\\{macro_name}{{{event}}}{{%\n{rows_block}\n}}"

        # Wrap in minipage
        width = 1 / ncols
        out.append(f"\\begin{{minipage}}{{{width:.3f}\\linewidth}}\\centering\n{latex_cmd}\n\\end{{minipage}}")

        # Row break every ncols
        if (i+1) % ncols == 0:
            out.append("\\par\\bigskip")


    print("\n".join(out))

# Connect button
print_btn.on_click(print_all_events)

# Display UI
display(sheet_dd, sex_dd, rows_dd, cols_dd, print_btn)




In [ ]:
# Strip whitespace from column names just in case
times_df.columns = times_df.columns.str.strip()

output = []

for sheet in sorted(times_df["SHEET"].dropna().unique()):

    output.append(f"\n\\section{{{sheet}}}")

    sexes = sorted(times_df.loc[times_df["SHEET"] == sheet, "SEX"].dropna().unique())
    for sex in sexes:
        output.append(f"\\subsection{{{sex}}}")


# Print LaTeX skeleton
print("\n".join(output))


In [ ]:
import re

# Clean event names
times_df["EVENT_clean"] = times_df["EVENT"].astype(str).str.strip()

# Normalize "3-M" → "3-METER" and remove double spaces
times_df["EVENT_clean"] = times_df["EVENT_clean"].str.replace(r'\b3-M\b', '3-METER', regex=True)
times_df["EVENT_clean"] = times_df["EVENT_clean"].str.replace(r'\s+', ' ', regex=True)

unique_events = times_df["EVENT_clean"].dropna().unique()

# Stroke order for individuals
stroke_order = ["FREE", "BACK", "BREAST", "FLY", "IM"]

def event_sort_key(event):
    event_upper = event.upper()

    # Check if diving
    if "METER" in event_upper:
        stroke_index = 999  # Diving goes last
        # Extract numeric meter for secondary sort
        match = re.search(r'\d+', event_upper)
        distance = int(match.group()) if match else 0
        # Check for "(X DIVES)" and use number for tertiary sort
        dives_match = re.search(r'\((\d+)\s*DIVES\)', event_upper)
        dives = int(dives_match.group(1)) if dives_match else 0
        return (stroke_index, distance, dives, event_upper)

    # Check if relay/split
    relay_flag = 1 if "RELAY" in event_upper or "SPL" in event_upper else 0

    # Determine stroke index for individual events
    stroke_index = 0
    for i, stroke in enumerate(stroke_order):
        if stroke in event_upper:
            stroke_index = i
            break

    # Extract numeric distance
    match = re.search(r'\d+', event_upper)
    distance = int(match.group()) if match else 0

    return (relay_flag, stroke_index, distance, event_upper)

# Sort unique events
sorted_events = sorted(set(unique_events), key=event_sort_key)

# Print result
for e in sorted_events:
    print(e)


In [28]:
import pandas as pd
import re

# Assuming times_df is already loaded
# times_df = pd.read_csv(...)

# 1. Strip whitespace from all string columns
str_cols = times_df.select_dtypes(include='object').columns
times_df[str_cols] = times_df[str_cols].apply(lambda x: x.str.strip())

# 2. Normalize column names (optional)
times_df.columns = [c.strip().upper() for c in times_df.columns]

# 3. Normalize events
times_df["EVENT"] = times_df["EVENT"].astype(str).str.strip()
times_df["EVENT"] = times_df["EVENT"].str.replace(r'\s+', ' ', regex=True)       # remove double spaces
times_df["EVENT"] = times_df["EVENT"].str.replace(r'\b3-M\b', '3-METER', regex=True)  # fix 3-M

# 4. Add clean event column for sorting
times_df["EVENT_CLEAN"] = times_df["EVENT"]

# 5. Define sorting key (same as before)
stroke_order = ["FREE", "BACK", "BREAST", "FLY", "IM"]

def event_sort_key(event):
    event_upper = event.upper()

    # Diving events
    if "METER" in event_upper:
        stroke_index = 999
        match = re.search(r'\d+', event_upper)
        distance = int(match.group()) if match else 0
        dives_match = re.search(r'\((\d+)\s*DIVES\)', event_upper)
        dives = int(dives_match.group(1)) if dives_match else 0
        return (stroke_index, distance, dives, event_upper)

    # Relay/Splits
    relay_flag = 1 if "RELAY" in event_upper or "SPL" in event_upper else 0

    # Individual strokes
    stroke_index = 0
    for i, stroke in enumerate(stroke_order):
        if stroke in event_upper:
            stroke_index = i
            break

    match = re.search(r'\d+', event_upper)
    distance = int(match.group()) if match else 0

    return (relay_flag, stroke_index, distance, event_upper)

# 6. Sort times_df by EVENT_CLEAN
times_df = times_df.sort_values("EVENT_CLEAN", key=lambda x: x.map(event_sort_key))

# 7. Reset index
times_df = times_df.reset_index(drop=True)


In [29]:

times_df.to_csv(OUT_FOLDER / "times_df.csv", index=False)


In [32]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import math

# Dropdowns
sheets = sorted(times_df["SHEET"].dropna().unique())
sexes = sorted(times_df["SEX"].dropna().unique())

sheet_dd = widgets.Dropdown(options=sheets, description="Sheet:")
sex_dd = widgets.Dropdown(options=sexes, description="Sex:")
rows_dd = widgets.IntSlider(value=3, min=1, max=5, description="Rows:")
cols_dd = widgets.IntSlider(value=3, min=1, max=5, description="Cols:")

print_btn = widgets.Button(description="Print Tables")

# Function to generate LaTeX tables in subtable grid
def print_event_grid(_):
    clear_output(wait=True)
    display(sheet_dd, sex_dd, rows_dd, cols_dd, print_btn)

    sheet = sheet_dd.value
    sex = sex_dd.value
    nrows, ncols = rows_dd.value, cols_dd.value

    df_filtered = times_df[(times_df["SHEET"] == sheet) & (times_df["SEX"] == sex)]
    if df_filtered.empty:
        print("⚠️ No events for this selection.")
        return

    # Sort unique events
    events = sorted(df_filtered["EVENT_CLEAN"].dropna().unique(), key=event_sort_key)

    # Generate LaTeX for each event table
    table_codes = []
    for event in events:
        df_event = df_filtered[df_filtered["EVENT_CLEAN"] == event]
        cols_used = [col for col in df_event.columns if col not in {"SHEET","SEX","EVENT","EVENT_CLEAN"} and df_event[col].notna().any()]
        macro_name = "eventtable" + "".join(col.capitalize() for col in cols_used)

        rows = []
        for _, row in df_event.iterrows():
            vals = [str(row[c]) for c in cols_used]
            rows.append(" & ".join(vals) + " \\\\")
        rows_block = "\n".join(rows)
        latex_cmd = f"\\{macro_name}{{{event}}}{{%\n{rows_block}\n}}"
        table_codes.append(latex_cmd)

    # Wrap tables in subtable grid
    out = []
    tables_per_page = nrows * ncols
    for page_start in range(0, len(table_codes), tables_per_page):
        page_tables = table_codes[page_start:page_start + tables_per_page]
        out.append("\\begin{table}[h!]\n\\centering")
        for i, t in enumerate(page_tables):
            out.append(f"\\begin{{subtable}}[t]{{{1/ncols:.3f}\\linewidth}}\\centering\n{t}\n\\end{{subtable}}")
            if (i + 1) % ncols != 0:
                out.append("%")  # prevents unwanted space between subtables
            else:
                out.append("\\\\[1em]")  # new row
        out.append(f"\\caption{{Events for Sheet {sheet}, Sex {sex}}}")
        out.append("\\end{table}\n\n")

    print("\n".join(out))

print_btn.on_click(print_event_grid)
display(sheet_dd, sex_dd, rows_dd, cols_dd, print_btn)


Dropdown(description='Sheet:', options=('CMS All Time Top 10', 'CMS Axelrood Pool Records', 'CMS Frosh Swimmin…

Dropdown(description='Sex:', options=('Athena', 'Men', 'Stag', 'Stags', 'Women'), value='Athena')

IntSlider(value=3, description='Rows:', max=5, min=1)

IntSlider(value=3, description='Cols:', max=5, min=1)

Button(description='Print Tables', style=ButtonStyle())

\begin{table}[h!]
\centering
\begin{subtable}[t]{0.333\linewidth}\centering
\eventtableTimeNameYear{50 FREE}{%
23.29 & Jocelyn Crawford & 2019 \\
23.31 & Ava Sealander & 2022 \\
23.49 & Kelly Ngo & 2016 \\
23.77 & Suzia Starzyk & 2020 \\
23.71 & Helen Liu & 2014 \\
23.75 & Annika Sharma & 2024 \\
23.76 & Michele Kee & 2014 \\
23.87 & Katie Bilotti & 2010 \\
23.77 & Natalia Orbach-M & 2020 \\
23.39 & Madeleine Kan & 2025 \\
}
\end{subtable}
%
\begin{subtable}[t]{0.333\linewidth}\centering
\eventtableTimeNameYear{100 FREE}{%
51.05 & Kelly Ngo & 2016 \\
51.24 & Jocelyn Crawford & 2017 \\
51.41 & Michele Kee & 2014 \\
51.47 & Madeleine Kan & 2025 \\
51.56 & Natalia Orbach-M & 2020 \\
51.72 & Annika Sharma & 2024 \\
51.88 & Riley Hoffman & 2019 \\
52.05 & Emma Jones & 2009 \\
52.05 & Jenni Rinker & 2011 \\
51.56 & Ava Sealander & 2022 \\
}
\end{subtable}
%
\begin{subtable}[t]{0.333\linewidth}\centering
\eventtableTimeNameYear{200 FREE}{%
1:50.30 & Ella Blake & 2022 \\
1:51.75 & Sun Young By

In [ ]:
# Check if the second CSV file exists
csv_file2 = OUT_FOLDER / "times_df - times_df.csv"
if csv_file2.exists():
    times_df2 = pd.read_csv(csv_file2)
    print(f"Loaded existing CSV file: {csv_file2}")
else:
    print(f"CSV file not found: {csv_file2}")
    print("You may need to create this file from your Excel data first.")
    # Create an empty dataframe for now
    times_df2 = pd.DataFrame()

times_df2.head()

,SHEET,SEX,EVENT,TIME,NAME,YEAR,TEAM,RANK,SITE,MEET,CONTEXT
0,CMS at PP,Athena,50 FREE,23.29,Jocelyn Crawford,2018,NaN,NaN,NaN,NaN,NaN
1,CMS at PP Combined,Athena,50 FREE,23.89,Ava Sealander,2021,CMS,NaN,NaN,NaN,NaN
2,CMS at PP Combined,Athena,50 FREE,23.85,Francesca Coppo,2023,PP,NaN,NaN,NaN,NaN
3,CMS at PP Combined,Athena,50 FREE,23.67,Madison Kauahi,2017,PP,NaN,NaN,NaN,NaN
4,CMS at PP Combined,Athena,50 FREE,23.66,Madeleine Kan,2024,CMS,NaN,NaN,NaN,NaN


In [36]:
# Print all unique sheet names
unique_sheets = times_df2["SHEET"].dropna().unique()
print(sorted(unique_sheets))


['CMS All Time Top 10', 'CMS Axelrood Pool Records', 'CMS Frosh Swimming & Diving Records', 'CMS SCIAC Champions', 'CMS at Cal Baptist Distance Meet', 'CMS at PP', 'CMS at PP Combined', 'CMS at UCSD', 'Development of Team Records (October 2001 to March 2025)', 'NCAA TOP 20', 'SCIAC All Time Top 10 Performers', 'SCIAC Records']


In [41]:
import pandas as pd
from pathlib import Path
import re

# Custom sheet and sex orders
sheet_order = [
    'CMS All Time Top 10', 'CMS Axelrood Pool Records', 'CMS Frosh Swimming & Diving Records',
    'Development of Team Records (October 2001 to March 2025)', 'CMS at UCSD',
    'CMS at Cal Baptist Distance Meet', 'CMS at PP', 'CMS at PP Combined',
    'CMS SCIAC Champions', 'SCIAC All Time Top 10 Performers', 'SCIAC Records', 'NCAA TOP 20'
]
sex_order = ["Women", "Athena", "Men", "Stag"]

# Convert sheet/sex to ordered categorical
times_df2["SHEET"] = pd.Categorical(times_df2["SHEET"], categories=sheet_order, ordered=True)
times_df2["SEX"] = pd.Categorical(times_df2["SEX"], categories=sex_order, ordered=True)

# Define event sort key as before
stroke_order = ["FREE", "BACK", "BREAST", "FLY", "IM"]

def event_sort_key(event):
    event_upper = event.upper()

    # Diving events
    if "METER" in event_upper:
        stroke_index = 999
        match = re.search(r'\d+', event_upper)
        distance = int(match.group()) if match else 0
        dives_match = re.search(r'\((\d+)\s*DIVES\)', event_upper)
        dives = int(dives_match.group(1)) if dives_match else 0
        return (stroke_index, distance, dives, event_upper)

    # Relay/Splits
    relay_flag = 1 if "RELAY" in event_upper or "SPL" in event_upper else 0

    # Individual strokes
    stroke_index = 0
    for i, stroke in enumerate(stroke_order):
        if stroke in event_upper:
            stroke_index = i
            break

    match = re.search(r'\d+', event_upper)
    distance = int(match.group()) if match else 0

    return (relay_flag, stroke_index, distance, event_upper)

# Convert EVENT to categorical based on custom key
unique_events = times_df2["EVENT"].dropna().unique()
sorted_events = sorted(unique_events, key=event_sort_key)
times_df2["EVENT"] = pd.Categorical(times_df2["EVENT"], categories=sorted_events, ordered=True)

# Sort by sheet → sex → event
times_df_sorted = times_df2.sort_values(by=["SHEET", "SEX", "EVENT"]).reset_index(drop=True)


times_df_sorted.to_csv(OUT_FOLDER / "times_df_sorted2.csv", index=False)


In [ ]:
# Check if the final sorted CSV file exists
final_csv_file = OUT_FOLDER / "times_df_sorted2.csv"
if final_csv_file.exists():
    final_times_df = pd.read_csv(final_csv_file)
    print(f"Loaded existing CSV file: {final_csv_file}")
else:
    print(f"CSV file not found: {final_csv_file}")
    print("You may need to create this file from your Excel data first.")
    # Create an empty dataframe for now
    final_times_df = pd.DataFrame()

final_times_df.head()

,SHEET,SEX,EVENT,TIME,NAME,YEAR,TEAM,RANK,SITE,MEET,CONTEXT
0,CMS All Time Top 10,Athena,50 FREE,23.29,Jocelyn Crawford,2019,NaN,NaN,NaN,NaN,NaN
1,CMS All Time Top 10,Athena,50 FREE,23.31,Ava Sealander,2022,NaN,NaN,NaN,NaN,NaN
2,CMS All Time Top 10,Athena,50 FREE,23.49,Kelly Ngo,2016,NaN,NaN,NaN,NaN,NaN
3,CMS All Time Top 10,Athena,50 FREE,23.77,Suzia Starzyk,2020,NaN,NaN,NaN,NaN,NaN
4,CMS All Time Top 10,Athena,50 FREE,23.71,Helen Liu,2014,NaN,NaN,NaN,NaN,NaN


In [62]:
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output

class SwimTableGenerator:
    def __init__(self, df):
        self.df = df
        self.selected_tables = []
        self.layout_config = {'per_row': 3, 'per_page': 9}

    def get_available_combinations(self):
        """Get all unique combinations of SHEET, SEX, EVENT in order they appear"""
        # Get unique combinations while preserving order
        seen = set()
        combinations_list = []

        for _, row in self.df.iterrows():
            combo = (row['SHEET'], row['SEX'], row['EVENT'])
            if combo not in seen:
                combinations_list.append({
                    'SHEET': row['SHEET'],
                    'SEX': row['SEX'],
                    'EVENT': row['EVENT']
                })
                seen.add(combo)

        combinations = pd.DataFrame(combinations_list)

        # Add count and display name
        for idx, row in combinations.iterrows():
            mask = ((self.df['SHEET'] == row['SHEET']) &
                   (self.df['SEX'] == row['SEX']) &
                   (self.df['EVENT'] == row['EVENT']))
            combinations.at[idx, 'count'] = mask.sum()

        combinations['display_name'] = (combinations['SHEET'] + ' - ' +
                                       combinations['SEX'] + ' - ' +
                                       combinations['EVENT'])
        return combinations

    def preview_table_data(self, sheet, sex, event):
        """Preview the data for a specific table"""
        mask = ((self.df['SHEET'] == sheet) &
                (self.df['SEX'] == sex) &
                (self.df['EVENT'] == event))
        return self.df[mask]

    def generate_table_latex(self, sheet, sex, event, use_inline=False):
        """Generate LaTeX for a single table"""
        data = self.preview_table_data(sheet, sex, event)
        title = event  # Only use the event name in caption

        # Sort by TIME (fastest to slowest)
        if 'TIME' in data.columns:
            # Convert TIME to numeric for proper sorting
            def time_to_seconds(time_str):
                if pd.isna(time_str):
                    return float('inf')
                try:
                    time_str = str(time_str).strip()
                    if ':' in time_str:
                        # Handle format like "1:50.30"
                        parts = time_str.split(':')
                        minutes = float(parts[0])
                        seconds = float(parts[1])
                        return minutes * 60 + seconds
                    else:
                        # Handle format like "23.29"
                        return float(time_str)
                except (ValueError, AttributeError):
                    return float('inf')

            data = data.copy()
            data['time_seconds'] = data['TIME'].apply(time_to_seconds)
            data = data.sort_values('time_seconds').drop('time_seconds', axis=1)
            data = data.reset_index(drop=True)  # Reset index after sorting

        # Determine which columns to use
        available_cols = []

        for col in ['TIME', 'NAME', 'YEAR', 'TEAM', 'RANK', 'SITE', 'MEET', 'CONTEXT']:
            if col in data.columns and not data[col].isna().all():
                available_cols.append(col)

        # Generate table rows
        rows = []
        for _, row in data.iterrows():
            row_data = []
            for col in available_cols:
                value = str(row[col]) if pd.notna(row[col]) else ""
                row_data.append(value)
            rows.append(" & ".join(row_data) + " \\\\")

        table_content = "\n".join(rows)

        # Generate macro name
        macro_suffix = "Mini" if use_inline else ""
        macro_name = f"\\eventtable{''.join(available_cols)}{macro_suffix}"

        return f"{macro_name}{{{title}}}{{\n{table_content}\n}}"

    def generate_layout_latex(self, layout_type='minipage'):
        """Generate complete LaTeX with layout"""
        if not self.selected_tables:
            return "No tables selected!"

        latex_output = []
        table_counter = 0
        per_row = self.layout_config['per_row']
        per_page = self.layout_config['per_page']

        if layout_type == 'minipage':
            minipage_width = f"{0.95/per_row:.2f}"

            for table_info in self.selected_tables:
                sheet, sex, event, _ = table_info

                # Start new row if needed
                if table_counter % per_row == 0:
                    if table_counter > 0:
                        latex_output.append("\\\\[0.5cm]")
                    latex_output.append("")

                # Use mini macro format for minipage
                table_latex = self.generate_table_latex(sheet, sex, event, use_inline=True)

                minipage = f"""\\begin{{minipage}}{{{minipage_width}\\textwidth}}
\\centering
{table_latex}
\\end{{minipage}}"""

                if table_counter % per_row < per_row - 1:
                    minipage += "\\hfill"

                latex_output.append(minipage)
                table_counter += 1

                # Page break if needed
                if table_counter % per_page == 0:
                    latex_output.append("\\newpage")

        elif layout_type == 'standalone':
            # Generate standalone tables using your original macros
            for table_info in self.selected_tables:
                sheet, sex, event, _ = table_info
                table_latex = self.generate_table_latex(sheet, sex, event, use_inline=False)
                latex_output.append(table_latex)
                latex_output.append("")  # Add space between tables

        elif layout_type == 'longtable':
            col_spec = "@{}" + "p{" + f"{0.9/per_row:.2f}" + "\\textwidth}" * per_row + "@{}"
            latex_output.append(f"\\begin{{longtable}}{{{col_spec}}}")

            tables_in_row = []
            for table_info in self.selected_tables:
                sheet, sex, event, _ = table_info
                table_latex = self.generate_table_latex(sheet, sex, event, use_inline=True)

                table_cell = f"\\parbox[t]{{{0.85/per_row:.2f}\\textwidth}}{{\\centering {table_latex}}}"
                tables_in_row.append(table_cell)

                if len(tables_in_row) == per_row:
                    latex_output.append(" & ".join(tables_in_row) + " \\\\[0.8cm]")
                    tables_in_row = []

            # Handle remaining tables
            if tables_in_row:
                while len(tables_in_row) < per_row:
                    tables_in_row.append("")
                latex_output.append(" & ".join(tables_in_row) + " \\\\")

            latex_output.append("\\end{longtable}")

        return "\n".join(latex_output)

# Initialize the generator
generator = SwimTableGenerator(final_times_df)

# Get available combinations
combinations = generator.get_available_combinations()
print("Available table combinations (in order of appearance):")
print(combinations[['display_name', 'count']].to_string(index=False))

Available table combinations (in order of appearance):
                                                                                 display_name  count
                                                       CMS All Time Top 10 - Athena - 50 FREE   10.0
                                                      CMS All Time Top 10 - Athena - 100 FREE   10.0
                                                      CMS All Time Top 10 - Athena - 200 FREE   10.0
                                                      CMS All Time Top 10 - Athena - 500 FREE   10.0
                                                     CMS All Time Top 10 - Athena - 1000 FREE   10.0
                                                     CMS All Time Top 10 - Athena - 1650 FREE   10.0
                                                      CMS All Time Top 10 - Athena - 100 BACK   10.0
                                                      CMS All Time Top 10 - Athena - 200 BACK   10.0
                                    

In [65]:
def create_table_interface():
    combinations = generator.get_available_combinations()

    # Widgets - dropdown maintains order from dataframe
    table_dropdown = widgets.Dropdown(
        options=[(row['display_name'], (row['SHEET'], row['SEX'], row['EVENT']))
                for _, row in combinations.iterrows()],
        description='Select Table:',
        style={'description_width': 'initial'}
    )

    add_button = widgets.Button(description='Add Table', button_style='success')
    preview_button = widgets.Button(description='Preview Data', button_style='info')

    # Layout configuration
    per_row_slider = widgets.IntSlider(
        value=2, min=1, max=4, description='Tables per row:',
        style={'description_width': 'initial'}
    )

    per_page_slider = widgets.IntSlider(
        value=6, min=3, max=16, description='Tables per page:',
        style={'description_width': 'initial'}
    )

    layout_dropdown = widgets.Dropdown(
        options=[
            ('Minipage (side by side)', 'minipage'),
            ('Standalone (your macros)', 'standalone'),
            ('Longtable (grid)', 'longtable')
        ],
        value='minipage',
        description='Layout type:',
        style={'description_width': 'initial'}
    )

    generate_button = widgets.Button(description='Generate LaTeX', button_style='primary')
    clear_button = widgets.Button(description='Clear Selection', button_style='warning')

    output = widgets.Output()

    def add_table(b):
        sheet, sex, event = table_dropdown.value
        generator.selected_tables.append((sheet, sex, event, False))

        with output:
            clear_output()
            print(f"Added: {sheet} - {sex} - {event}")
            print(f"Total selected: {len(generator.selected_tables)}")
            print("\nCurrent selection:")
            for i, (s, sx, e, _) in enumerate(generator.selected_tables, 1):
                print(f"{i}. {s} - {sx} - {e}")

    def preview_data(b):
        sheet, sex, event = table_dropdown.value
        data = generator.preview_table_data(sheet, sex, event)

        # Sort the preview data too
        if 'TIME' in data.columns:
            def time_to_seconds(time_str):
                if pd.isna(time_str):
                    return float('inf')
                try:
                    time_str = str(time_str).strip()
                    if ':' in time_str:
                        parts = time_str.split(':')
                        minutes = float(parts[0])
                        seconds = float(parts[1])
                        return minutes * 60 + seconds
                    else:
                        return float(time_str)
                except (ValueError, AttributeError):
                    return float('inf')

            data = data.copy()
            data['time_seconds'] = data['TIME'].apply(time_to_seconds)
            data = data.sort_values('time_seconds')
            # Show the time_seconds for debugging
            print("Debug - time conversion (first 5):")
            for _, row in data.head().iterrows():
                print(f"  {row['TIME']} -> {row['time_seconds']}")
            data = data.drop('time_seconds', axis=1).reset_index(drop=True)

        with output:
            clear_output()
            print(f"Preview: {sheet} - {sex} - {event} (sorted by time)")
            print(f"Rows: {len(data)}")
            display(data)

    def generate_latex(b):
        generator.layout_config['per_row'] = per_row_slider.value
        generator.layout_config['per_page'] = per_page_slider.value

        latex_code = generator.generate_layout_latex(layout_dropdown.value)

        with output:
            clear_output()
            print(latex_code)

    def clear_selection(b):
        generator.selected_tables = []
        with output:
            clear_output()
            print("Selection cleared!")

    # Connect buttons
    add_button.on_click(add_table)
    preview_button.on_click(preview_data)
    generate_button.on_click(generate_latex)
    clear_button.on_click(clear_selection)

    # Layout
    table_controls = widgets.HBox([table_dropdown])
    action_buttons = widgets.HBox([add_button, preview_button, clear_button])
    layout_controls = widgets.VBox([
        widgets.Label("Layout Configuration:"),
        per_row_slider,
        per_page_slider,
        layout_dropdown,
        generate_button
    ])

    interface = widgets.VBox([
        widgets.Label("Swimming Table Generator"),
        table_controls,
        action_buttons,
        layout_controls,
        output
    ])

    return interface

# Create and display the interface
interface = create_table_interface()
display(interface)